# Current Work

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os
import json

# notebook settings
pd.set_option('display.max_columns', None)

In [4]:
path = "./unsynced-data/crops-1112023/"
crop_files = os.listdir(path)
crop_files = [f for f in crop_files if f.endswith('.csv')]
grid = gpd.read_file("./synced-data/empty-grid.geojson")


In [5]:
def reduce_simulations(df, string, grid_df):
    working_df = df[["Exp.", "Yield", "Biomass", "Duration", "lat", "long"]].groupby("Exp.").mean().reset_index()
    # clean up and turn df into geodataframe
    working_df.columns = map(str.lower, working_df.columns)
    working_df = working_df.rename(columns={"yield": "yield_" + string, "biomass": "biomass_" + string, "duration": "duration_" + string})
    working_df = gpd.GeoDataFrame(working_df, crs="EPSG:4326", geometry=gpd.points_from_xy(working_df['long'], working_df['lat']))
    working_df = working_df.drop(columns=["lat", "long", "exp."])
    # add working_df to grid_df where geometries intersect on geometry
    grid_df = gpd.sjoin(grid_df, working_df, how="left", op="intersects")
    grid_df = grid_df.drop(columns='index_right')
    grid_df.columns = map(str.lower, grid_df.columns)
    return grid_df

In [6]:
def reduce_models(df):
    # create a list of column names
    column_names = df.columns.tolist()
    column_names.remove("geometry")
    models = ["_ipsl", "_mpi", "_gfdl", "_mri"]
    
    for m in models:
        # remove  m from the column names
        column_names = [c.replace(m, "") for c in column_names]
    # remove duplicates in column_names list by converting to set and back to list
    column_names = list(set(column_names))
    # average all columns that match a column name
    for c in column_names:
        df[c] = df.filter(regex=c).mean(axis=1)
    # drop all columns that include a string from models
    df = df.drop(columns=[c for c in df.columns if any(m in c for m in models)])
    df = df.round(3)
    return df


In [7]:
# It feels like there should be a better way to remove/not write NAN's to a geojson file to save space

def geojson_no_nans(df, path):
    # First write a geopandas dataframe to a geojson file
    df.to_file(path, driver='GeoJSON')

    # Load the GeoJSON data from a file
    with open(path, 'r') as geojson_file:
        geojson_data = json.load(geojson_file)

    def remove_null_properties(feature):
        non_null_properties = {key: value for key, value in feature['properties'].items() if value is not None}
        feature['properties'] = non_null_properties

    # Loop through features and remove null properties
    for feature in geojson_data['features']:
        remove_null_properties(feature)

    # Save the modified GeoJSON to a new file
    with open(path, 'w') as output_file:
        json.dump(geojson_data, output_file, indent=2)

In [8]:
# a dictionary of column quanitiles is useful for styling the map in the frontend
def generate_quantiles(df):
    col_list = df.columns.tolist()
    col_list.remove("geometry")
    q_list = ["min", 0.01, 0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.98, 0.99, "max"]
    working_dict = {}
    for col in col_list:
        quantile_dict = {}
        for quantile in q_list:
            if quantile == "min":
                quantile_value = df[col].min()
                quantile_dict[quantile] = quantile_value

            elif quantile == "max":
                quantile_value = df[col].max()
                quantile_dict[quantile] = quantile_value
            else:
                quantile_value = df[col].quantile(quantile)
                quantile_dict[f"quantile_{int(quantile * 100)}"] = quantile_value  
        working_dict[col] = quantile_dict
    return working_dict

In [9]:
for c in crop_files:
    file_df = pd.read_csv(path + c)
    column_string = c.split("-")[0]
    grid = reduce_simulations(file_df, column_string, grid)
grid = grid.round(3)


/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/inter

In [18]:
# correcting column names removing the reference to a simulation in the historical column names
for c in grid.columns:
    if "historical" in c:
        grid[c.replace("_ssp370", "")] = grid[c]
        grid = grid.drop(columns=c)


/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: PerformanceWarning: DataFrame is highly fragmented.  T

In [19]:
grid

geometry  yield_taro_future_ssp370_mpi   
0      POINT (19.25000 -34.25000)                           NaN  \
1      POINT (19.75000 -34.25000)                           NaN   
2      POINT (20.25000 -34.25000)                           NaN   
3      POINT (20.75000 -34.25000)                           NaN   
4      POINT (21.25000 -34.25000)                           NaN   
...                           ...                           ...   
10230    POINT (9.25000 36.75000)                           NaN   
10231    POINT (9.75000 36.75000)                           NaN   
10232   POINT (10.25000 36.75000)                           NaN   
10233   POINT (10.75000 36.75000)                           NaN   
10234    POINT (9.75000 37.25000)                           NaN   

       biomass_taro_future_ssp370_mpi  duration_taro_future_ssp370_mpi   
0                                 NaN                              NaN  \
1                                 NaN                              NaN   
2                                 NaN                              NaN   
3                                 NaN                              NaN   
4                                 NaN                              NaN   
...                               ...                              ...   
10230                             NaN                              NaN   
10231                             NaN                              NaN   
10232                             NaN                              NaN   
10233                             NaN                              NaN   
10234                             NaN                              NaN   

       yield_fonio_future_ssp126_mri  biomass_fonio_future_ssp126_mri   
0                                NaN                              NaN  \
1                                NaN                              NaN   
2                                NaN                              NaN   
3                                NaN                              NaN   
4                                NaN                              NaN   
...                              ...                              ...   
10230                            NaN                              NaN   
10231                            NaN                              NaN   
10232                            NaN                              NaN   
10233                            NaN                              NaN   
10234                            NaN                              NaN   

       duration_fonio_future_ssp126_mri   
0                                   NaN  \
1                                   NaN   
2                                   NaN   
3                                   NaN   
4                                   NaN   
...                                 ...   
10230                               NaN   
10231                               NaN   
10232                               NaN   
10233                               NaN   
10234                               NaN   

       yield_fingermillet_future_ssp126_ipsl   
0                                        NaN  \
1                                        NaN   
2                                        NaN   
3                                        NaN   
4                                        NaN   
...                                      ...   
10230                                    NaN   
10231                                    NaN   
10232                                    NaN   
10233                                    NaN   
10234                                    NaN   

       biomass_fingermillet_future_ssp126_ipsl   
0                                          NaN  \
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
...                                        ...   
10230     

In [20]:
geojson_no_nans(grid, "./synced-data/crop-yields-all-models.geojson")
grid.to_csv("./synced-data/crop-yields-all-models.csv", index=False) 

In [11]:
grid

geometry  yield_okra_historical_ssp370_mri   
0      POINT (19.25000 -34.25000)                               NaN  \
1      POINT (19.75000 -34.25000)                               NaN   
2      POINT (20.25000 -34.25000)                               NaN   
3      POINT (20.75000 -34.25000)                               NaN   
4      POINT (21.25000 -34.25000)                               NaN   
...                           ...                               ...   
10230    POINT (9.25000 36.75000)                               NaN   
10231    POINT (9.75000 36.75000)                               NaN   
10232   POINT (10.25000 36.75000)                               NaN   
10233   POINT (10.75000 36.75000)                               NaN   
10234    POINT (9.75000 37.25000)                               NaN   

       biomass_okra_historical_ssp370_mri   
0                                     NaN  \
1                                     NaN   
2                                     NaN   
3                                     NaN   
4                                     NaN   
...                                   ...   
10230                                 NaN   
10231                                 NaN   
10232                                 NaN   
10233                                 NaN   
10234                                 NaN   

       duration_okra_historical_ssp370_mri  yield_taro_future_ssp370_mpi   
0                                      NaN                           NaN  \
1                                      NaN                           NaN   
2                                      NaN                           NaN   
3                                      NaN                           NaN   
4                                      NaN                           NaN   
...                                    ...                           ...   
10230                                  NaN                           NaN   
10231                                  NaN                           NaN   
10232                                  NaN                           NaN   
10233                                  NaN                           NaN   
10234                                  NaN                           NaN   

       biomass_taro_future_ssp370_mpi  duration_taro_future_ssp370_mpi   
0                                 NaN                              NaN  \
1                                 NaN                              NaN   
2                                 NaN                              NaN   
3                                 NaN                              NaN   
4                                 NaN                              NaN   
...                               ...                              ...   
10230                             NaN                              NaN   
10231                             NaN                              NaN   
10232                             NaN                              NaN   
10233                             NaN                              NaN   
10234                             NaN                              NaN   

       yield_fonio_future_ssp126_mri  biomass_fonio_future_ssp126_mri   
0                                NaN                              NaN  \
1                                NaN                              NaN   
2                                NaN                              NaN   
3                                NaN                              NaN   
4                                NaN                              NaN   
...                              ...                              ...   
10230                            NaN                              NaN   
10231                            NaN                              NaN   
10232                            NaN                              NaN   
10233                            NaN                              NaN   
10234                     

In [21]:
grid_reduced = reduce_models(grid)

In [22]:
grid_reduced

,geometry,duration_cowpea_future_ssp126,duration_cocoyam_future_ssp126,biomass_taro_future_ssp126,yield_fingermillet_future_ssp126,biomass_yams_future_ssp370,yield_africaneggplant_future_ssp126,biomass_bambaragroundnut_future_ssp126,yield_soybean_future_ssp370,duration_taro_future_ssp126,duration_lablab_future_ssp126,biomass_groundnut_future_ssp126,duration_bambaragroundnut_future_ssp370,biomass_lablab_future_ssp370,yield_cassava_future_ssp370,biomass_fingermillet_future_ssp370,yield_tef_future_ssp126,biomass_cassava_future_ssp126,duration_tomato_future_ssp370,yield_grasspea_future_ssp126,biomass_taro_future_ssp370,biomass_pigeonpea_future_ssp370,yield_fonio_future_ssp126,biomass_sesame_future_ssp370,duration_cowpea_future_ssp370,duration_groundnut_future_ssp370,duration_cocoyam_future_ssp370,yield_cowpea_future_ssp126,duration_africaneggplant_future_ssp370,duration_yams_future_ssp126,duration_cassava_future_ssp370,duration_sweetpotato_future_ssp126,yield_sesame_future_ssp126,duration_lablab_future_ssp370,duration_africaneggplant_future_ssp126,duration_yams_future_ssp370,duration_sesame_future_ssp370,biomass_groundnut_future_ssp370,biomass_sweetpotato_future_ssp370,biomass_sweetpotato_future_ssp126,biomass_sesame_future_ssp126,biomass_bambaragroundnut_future_ssp370,duration_bambaragroundnut_future_ssp126,biomass_grasspea_future_ssp370,biomass_tomato_future_ssp370,duration_fonio_future_ssp370,biomass_maize_future_ssp126,yield_sweetpotato_future_ssp370,yield_africaneggplant_future_ssp370,duration_fingermillet_future_ssp126,biomass_okra_future_ssp126,yield_bambaragroundnut_future_ssp126,yield_cocoyam_future_ssp370,duration_maize_future_ssp370,duration_taro_future_ssp370,duration_okra_future_ssp370,yield_cassava_future_ssp126,biomass_soybean_future_ssp126,yield_taro_future_ssp370,biomass_cassava_future_ssp370,yield_josephscoat_future_ssp126,yield_sorghum_future_ssp126,biomass_lablab_future_ssp126,yield_maize_future_ssp370,yield_cowpea_future_ssp370,yield_tomato_future_ssp126,yield_grasspea_future_ssp370,yield_groundnut_future_ssp126,biomass_pigeonpea_future_ssp126,duration_sorghum_future_ssp370,biomass_yams_future_ssp126,biomass_josephscoat_future_ssp370,duration_grasspea_future_ssp370,duration_grasspea_future_ssp126,biomass_sorghum_future_ssp126,biomass_tomato_future_ssp126,yield_okra_future_ssp370,yield_bambaragroundnut_future_ssp370,biomass_josephscoat_future_ssp126,duration_soybean_future_ssp370,duration_tef_future_ssp126,duration_mungbean_future_ssp370,duration_josephscoat_future_ssp126,yield_pigeonpea_future_ssp370,duration_soybean_future_ssp126,biomass_tef_future_ssp370,biomass_tef_future_ssp126,yield_sweetpotato_future_ssp126,yield_pigeonpea_future_ssp126,biomass_mungbean_future_ssp126,duration_pigeonpea_future_ssp126,biomass_grasspea_future_ssp126,biomass_cowpea_future_ssp370,biomass_fonio_future_ssp370,duration_mungbean_future_ssp126,yield_okra_future_ssp126,duration_sweetpotato_future_ssp370,duration_josephscoat_future_ssp370,biomass_africaneggplant_future_ssp126,biomass_cocoyam_future_ssp370,yield_tef_future_ssp370,biomass_maize_future_ssp370,duration_maize_future_ssp126,duration_groundnut_future_ssp126,yield_tomato_future_ssp370,yield_yams_future_ssp126,yield_yams_future_ssp370,biomass_africaneggplant_future_ssp370,duration_fingermillet_future_ssp370,yield_groundnut_future_ssp370,yield_mungbean_future_ssp126,yield_josephscoat_future_ssp370,biomass_sorghum_future_ssp370,biomass_fonio_future_ssp126,biomass_cocoyam_future_ssp126,yield_sorghum_future_ssp370,yield_mungbean_future_ssp370,biomass_okra_future_ssp370,yield_lablab_future_ssp126,biomass_soybean_future_ssp370,yield_fingermillet_future_ssp370,biomass_mungbean_future_ssp370,duration_cassava_future_ssp126,yield_fonio_future_ssp370,duration_sesame_future_ssp126,yield_cocoyam_future_ssp126,yield_lablab_future_ssp370,duration_tef_future_ssp370,duration_sorghum_future_ssp126,duration_fonio_future_ssp126,duration_pigeonpea_future_ssp370,duration_okra_future_ssp126,yield_

In [23]:
geojson_no_nans(grid_reduced, "./synced-data/crop-yields-mean-models.geojson")
grid_reduced.to_csv("./synced-data/crop-yields-mean-models.csv", index=False)

In [24]:
quantiles = generate_quantiles(grid_reduced)
# save to a json file data-quantiles.json
with open('./synced-data/data-quantiles.json', 'w') as fp:
    json.dump(quantiles, fp)
quantiles

{'duration_cowpea_future_ssp126': {'min': 16.766,
  'quantile_1': 18.35,
  'quantile_2': 19.483,
  'quantile_10': 27.292,
  'quantile_20': 43.25,
  'quantile_30': 54.225,
  'quantile_40': 60.543,
  'quantile_50': 65.69,
  'quantile_60': 69.683,
  'quantile_70': 72.215,
  'quantile_80': 75.384,
  'quantile_90': 80.667,
  'quantile_98': 88.536,
  'quantile_99': 92.058,
  'max': 110.8},
 'duration_cocoyam_future_ssp126': {'min': 58.65,
  'quantile_1': 63.0785,
  'quantile_2': 68.86099999999999,
  'quantile_10': 91.0586,
  'quantile_20': 111.0,
  'quantile_30': 116.4868,
  'quantile_40': 119.55640000000001,
  'quantile_50': 120.25,
  'quantile_60': 120.7032,
  'quantile_70': 120.958,
  'quantile_80': 132.9136,
  'quantile_90': 140.4786,
  'quantile_98': 151.93300000000002,
  'quantile_99': 154.33224,
  'max': 231.816},
 'biomass_taro_future_ssp126': {'min': 11.158,
  'quantile_1': 99.48701999999999,
  'quantile_2': 168.298,
  'quantile_10': 573.8752000000001,
  'quantile_20': 1201.77440000